# Summer Student 2025 Analysis - PEC

In [1]:
%%capture
%matplotlib widget
#!pip install requests_cache

import matplotlib.pyplot as plt
import time
import requests
import pandas as pd
import numpy as np
import datetime

import plotly.graph_objects as go
from scipy.signal import find_peaks

import ipywidgets as widgets
from IPython.display import display

import sys
sys.path.insert(1, '../python-scripts-c6fxKDJrSsWp1xCxON1Y7g')
sys.path.insert(1, '../../python-scripts-c6fxKDJrSsWp1xCxON1Y7g')
from api_calls import *
from pec_activity_stability_functions import *

url = "https://nomad-hzb-ce.de/nomad-oasis/api/v1"

import os
token = os.environ['NOMAD_CLIENT_ACCESS_TOKEN']

In [2]:
def get_ids_of_all_samples(url, token, author_name, sample_type='CE_NOME_Sample'):
    row = {"entry_type":sample_type,
           #"authors.name": "Abdelrahman Ishmael"
           #"authors.name": "Sofia Muñoz"
           #"authors.name": "Felipe Mata mata"
           "authors.name": author_name
        }
    query = {
        'required': {
            'metadata': '*',
            'data': '*',
        },
        'owner': 'visible',
        'query': row,
        'pagination': {
            'page_size': 100
        }
    }
    response = requests.post(f'{url}/entries/archive/query',
                             headers={'Authorization': f'Bearer {token}'}, json=query)
    return [s["archive"]["data"].get("lab_id") for s in response.json()["data"] if s["archive"]["data"].get("lab_id")]

def get_overview_table(sample_id_list):
    columns = ["Sample_id", "Bi(NO3)3 5H2O", "KI", "LA", "VO(acac)2", "benzoquinone", "G1_mAcm_2", "G_time", "N1_mAcm_2", "N_time", "NG_time", "no_cycles"]
    overview_table = pd.DataFrame(columns=columns)
    for sample_id in sample_id_list:
        try:
            row = {"Sample_id":sample_id}
            data = get_specific_data_of_sample(url, token, sample_id, "", with_meta=True)            
            data.sort(key=lambda x: datetime.datetime.strptime(x[0].get('datetime'), "%Y-%m-%dT%H:%M:%S.%f+00:00") if "." in x[0].get('datetime') else
             datetime.datetime.strptime(x[0].get('datetime'), "%Y-%m-%dT%H:%M:%S+00:00")       
             )
    
            N = data[0]
            G = data[1]
            NG = data[2]
            env = get_environment(url, token,N[1]["entry_id"])
            for s in env["substances"]:
                if "Bi(NO3)3 5H2O" in s["name"] or "BiN" in s["name"] or "Bi(NO3)3 x 5H2O" in s["name"]:
                    row.update({"Bi(NO3)3 5H2O":s["concentration_mmol_per_l"]})
                if "KI" in s["name"]:
                    row.update({"KI":s["concentration_mmol_per_l"]})
                if "Lactic Acid" in s["name"] or "Lactic" in s["name"] or "LA" in s["name"]:
                    row.update({"LA":s["concentration_mmol_per_l"]})
                if "Benzoquinone" in s["name"] or "benzo" in s["name"]:
                    row.update({"benzoquinone":s["concentration_mmol_per_l"]})
                if "VO(acac)2" in s["name"]:
                    row.update({"VO(acac)2":s["concentration_mmol_per_l"]})
            sample_area = N[0]["properties"]["sample_area"] / 1000 #mAcm_2
            row.update({"N1_mAcm_2":N[0]["properties"]["step_1_current"]/sample_area})
            row.update({"N_time":N[0]["properties"]["step_1_time"]})
            row.update({"no_cycles":(len(data)-1) // 2})
            sample_area = G[0]["properties"]["sample_area"] / 1000 #mAcm_2
            row.update({"G1_mAcm_2":G[0]["properties"]["step_1_current"]/sample_area})
            row.update({"G_time":G[0]["properties"]["step_1_time"]})
            row.update({"NG_time":NG[0]["properties"]["step_1_time"]})
            overview_table.loc[len(overview_table)] = row
        except:
            if sample_id != 'CE-NOME_FeMa_250709_0001':
                print(sample_id, "didnt work")
    return overview_table

def get_nife_deposition_params(sample_id_list):
    nife_dict = {}
    for sample_id in sample_id_list:
        data = get_specific_data_of_sample(url, token, sample_id, 'CE_NOME_Chronopotentiometry', with_meta=True)
        nife_env = []
        for cp in data:
            data, metadata = cp
            if metadata.get('upload_name').startswith('NiFe'):
                env_data = get_environment(url, token, metadata.get('entry_id'))
                env = {}
                for s in env_data.get('substances'):
                    if "Ni(NO3)2 6H2O" in s["name"]:
                        env['ni'] = s["concentration_mmol_per_l"]
                    if "FeCl2 4H2O" in s["name"]:
                        env['fe'] = s["concentration_mmol_per_l"]
                nife_env.append(env)
        if nife_env:
            nife_dict[sample_id] = nife_env
    return nife_dict

In [3]:
# all ipywidgets

author_selector = widgets.Dropdown(
    options=['Felipe Mata mata', 'Maddalena Zoli'],
    value='Felipe Mata mata',
    description='NOMAD author:',
    style={'description_width': 'initial'}
)

sample_selector = widgets.Dropdown(
    description="Select SampleID:",
    style={'description_width': 'initial'}
)

parameter_selector = widgets.Dropdown(
    options=['BiVO4 electrodeposition ("fast")', 'synthesis BiVO4 and NiFe ("slow")'],
    value=None,
    description='Select parameter set:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px'),
)

get_button = widgets.Button(
    description='Get NOMAD data',
    button_style='success',
    layout=widgets.Layout(width='auto')
)

analysis_button = widgets.Button(
    description='Analyse PEC data',
    button_style='info',
    layout=widgets.Layout(width='auto')
)


save_baybe_button = widgets.Button(
    description='Save data for Bayesian Optimization',
    button_style='primary',
    layout=widgets.Layout(width='auto')
)

show_sample_details_button = widgets.Button(
    description="Show plots for selected sample ID",
    button_style='info',
    layout=widgets.Layout(width='auto')
)

all_runs_output = widgets.Output()
analysis_output = widgets.Output()
baybe_output = widgets.Output()
sample_select_output = widgets.Output()
sample_detail_output = widgets.Output()

out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()

tab = widgets.Tab(children=[out1, out2, out3])
tab.set_title(0, 'Activity')
tab.set_title(1, 'Stability')
tab.set_title(2, 'Images')

In [4]:
def on_author_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        all_runs_output.clear_output()
        analysis_output.clear_output()
        baybe_output.clear_output()
        sample_select_output.clear_output()
        sample_detail_output.clear_output()

author_selector.observe(on_author_change)

display(author_selector)

Dropdown(description='NOMAD author:', options=('Felipe Mata mata', 'Maddalena Zoli'), style=DescriptionStyle(d…

## Show overview of all samples

In [5]:
overview_table = pd.DataFrame()

def on_button_clicked(b):
    global samples, overview_table
    with all_runs_output:
        all_runs_output.clear_output()
        analysis_output.clear_output()
        print('Getting data. This can take some time...')
        samples = get_ids_of_all_samples(url, token, author_selector.value)
        samples = sorted(samples)
        overview_table = get_overview_table(samples)
        samples = np.array(overview_table['Sample_id'])
        nife_dict = get_nife_deposition_params(samples)
        all_runs_output.clear_output()
        for sample_id, element_list in nife_dict.items():
            if len(element_list) > 1:
                print('Found more than one NiFe deposition for sample {sample_id}')
            if sample_id in overview_table["Sample_id"].values:
                overview_table.loc[overview_table["Sample_id"] == sample_id, "Ni"] = element_list[0]["ni"]
                overview_table.loc[overview_table["Sample_id"] == sample_id, "Fe"] = element_list[0]["fe"]
        display(overview_table)

get_button.on_click(on_button_clicked)

display(get_button, all_runs_output)

Button(button_style='success', description='Get NOMAD data', layout=Layout(width='auto'), style=ButtonStyle())

Output()

## Analyse PEC for given samples

Note: activity values of -1 mean, that the algorithm could not find an activity value. In the future we can calculate these values individually and set them manually.

In [6]:
def on_analysis_clicked(b):
    if overview_table.empty:
        with analysis_output:
            analysis_output.clear_output()
            print('Please use the "Get NOMAD data" button first.')
            return
    
    with analysis_output:
        analysis_output.clear_output()
        print('Evaluating data. This can take some time...')
        for sample_id in samples:
            entry_id = get_entryid(url, token, sample_id)
            description = get_entry_data(url, token, entry_id).get('description')
            overview_table.loc[overview_table['Sample_id']==sample_id, 'description'] = description
        
            if description and ('no deposition' in description or 'no good deposition' in description):
                overview_table.loc[overview_table['Sample_id']==sample_id, 'activity'] = 0
                overview_table.loc[overview_table['Sample_id']==sample_id, 'stability'] = 100
        
            photoactivity_entries = get_photoactivity_data_for_samples(url, token, [entry_id])
            for ca_data in photoactivity_entries:
                time = np.array(ca_data['archive']['data'].get('time'))
                current = np.array(ca_data['archive']['data'].get('current'))
                photoactivity_results = analyse_photoactivity(current, time)
                overview_table.loc[overview_table['Sample_id']==sample_id, 'activity'] = photoactivity_results['mean_delta_I']

            stability_entries = get_stability_data_for_samples(url, token, [entry_id])
            for entry in stability_entries:
                time = np.array(entry['archive']['data'].get('time'))
                voltage = np.array(entry['archive']['data'].get('voltage'))
                stability_analysis_result = analyse_stability(time, voltage)
                overview_table.loc[overview_table['Sample_id']==sample_id, 'stability'] = stability_analysis_result['percentage_of_loss']
                
        analysis_output.clear_output()
        display(overview_table)

    if not overview_table.empty:
        sample_selector.options = list(overview_table['Sample_id'])
        with sample_select_output:
            sample_select_output.clear_output()
            display(sample_selector)

analysis_button.on_click(on_analysis_clicked)

display(analysis_button, analysis_output)

Button(button_style='info', description='Analyse PEC data', layout=Layout(width='auto'), style=ButtonStyle())

Output()

## Inspect individual samples

In [7]:
def get_image_list(sample_id):
    data = get_specific_data_of_sample(url, token, sample_id, "CE_NOME_Measurement", with_meta=True)
    image_path_list = []
    for nomad_entry in data:
        data, metadata = nomad_entry
        file_name = data.get('data_file')[0]
        upload_name = metadata.get('upload_name')
        upload_id = metadata.get('upload_id')
        file_path = f'../../{upload_name.lower().replace(' ', '-')}-{upload_id}/{file_name}'
        image_path_list.append(file_path)
    return image_path_list

In [8]:
def on_sample_id_change(change):
    if change['name'] == 'value' and change['new'] is not None:
        sample_id = change['new']
        entry_id = get_entryid(url, token, sample_id)
        with sample_select_output:
            sample_select_output.clear_output()
            sample_detail_output.clear_output()
            display(sample_selector, overview_table[overview_table['Sample_id']==sample_id])
        with out1:
            out1.clear_output()
            photoactivity_entries = get_photoactivity_data_for_samples(url, token, [entry_id])
            for ca_data in photoactivity_entries:
                upload_name = ca_data['archive']['metadata'].get('upload_name', '')
                if sample_id in upload_name:
                    time = np.array(ca_data['archive']['data'].get('time'))
                    current = np.array(ca_data['archive']['data'].get('current'))
                    photoactivity_results = analyse_photoactivity(current, time)
                    print(f'{upload_name}, Mean ΔI: {photoactivity_results['mean_delta_I']}')
                    fig = get_photoactivity_plot(time, current, photoactivity_results)
                    fig.show()
        with out2:
            out2.clear_output()
            stability_entries = get_stability_data_for_samples(url, token, [entry_id])
            for entry in stability_entries:
                time = np.array(entry['archive']['data'].get('time'))
                voltage = np.array(entry['archive']['data'].get('voltage'))
                stability_analysis_result = analyse_stability(time, voltage)
                fig = get_stability_plot(time, voltage, stability_analysis_result)
                fig.show()
        with out3:
            out3.clear_output()
            image_paths = get_image_list(sample_id)
            #box = widgets.HBox([widgets.Image(value=open(b, 'rb').read(), format='png', width=200) for b in image_paths])
            box = widgets.HBox([widgets.Image(value=open(b, 'rb').read(), format='png') for b in image_paths])
            display(box)
        with sample_detail_output:
            sample_detail_output.clear_output()
            display(tab)

sample_selector.observe(on_sample_id_change, names='value')

if 'activity' not in overview_table:
    with sample_select_output:
        print('Please run the "Analyse PEC data" button before inspecting individual parameter sets.')

display(sample_select_output, sample_detail_output)


Output()

Output()

## Save Data for Bayesian Optimization

In [9]:
def save_deposition_baybe_table_csv(overview_df, author_name='felipe', target_col='stability'):
    rename_baybe = {
        'N_time': 'N1_s',
        'G_time': 'G1_s',
        'NG_time': 'NG1_s',
    }
    baybe_table = overview_df.rename(columns=rename_baybe)[['N1_mAcm_2', 'G1_mAcm_2', 'N1_s', 'G1_s', 'NG1_s', 'no_cycles', target_col]]
    #baybe_table = baybe_table[baybe_table[target_col].notna()]
    baybe_table.dropna(inplace=True)
    file_name = f'baybe_csv/bivo4_electrodeposition_{author_name}.csv'
    baybe_table.to_csv(file_name, index=False, header=True)
    print(f'Saved table for Bayesion Optimization in {file_name}')
    display(baybe_table)

def save_synthesis_baybe_table_csv(overview_df, author_name='felipe', target_col='stability'):
    rename_baybe = {
        'Bi(NO3)3 5H2O': 'BiNO3_M',
        'KI': 'KI_M',
        'LA': 'LA_M',
        'benzoquinone': 'benzoquinone_M',
        'VO(acac)2': 'Vacac2_M',
        'Ni': 'Ni_M',
    }
    cols = ['BiNO3_M', 'KI_M', 'LA_M', 'benzoquinone_M', 'Vacac2_M', 'Ni_M']
    baybe_table = overview_df.rename(columns=rename_baybe)[cols + [target_col]]
    baybe_table[cols] = baybe_table[cols] / 1000
    #baybe_table = baybe_table[baybe_table[target_col].notna()]
    baybe_table.dropna(inplace=True)
    baybe_table.to_csv(f'baybe_csv/bivo4_synthesis_{author_name}.csv', index=False, header=True)
    print(f'Saved table for Bayesion Optimization in baybe_csv/bivo4_synthesis_{author_name}.csv')
    display(baybe_table)

In [10]:
parameter_selector.value = 'BiVO4 electrodeposition ("fast")'

def on_baybe_clicked(b):
    with baybe_output:
        baybe_output.clear_output()
        if 'activity' not in overview_table or overview_table.empty:
            print('Please run the "Analyse PEC data" button before saving the table to csv.')
        else:
            first_name = author_selector.value.split()[0].lower()
            if parameter_selector.value == 'BiVO4 electrodeposition ("fast")':
                save_deposition_baybe_table_csv(overview_table, author_name=first_name, target_col='stability')
            elif parameter_selector.value == 'synthesis BiVO4 and NiFe ("slow")':
                save_synthesis_baybe_table_csv(overview_table, author_name=first_name, target_col='stability')

save_baybe_button.on_click(on_baybe_clicked)

display(parameter_selector, save_baybe_button, baybe_output)

Dropdown(description='Select parameter set:', layout=Layout(width='500px'), options=('BiVO4 electrodeposition …

Button(button_style='primary', description='Save data for Bayesian Optimization', layout=Layout(width='auto'),…

Output()